In [1]:
import os
import json
from IPython.display import JSON

# Poorly named, custom stuff
from wrappers import ytdlp_extract_audio, transcribe, summarize_key_features

In [2]:
DATA_ROOT = os.path.join(os.getcwd(), 'data')
AUDIO_OUTPUT = os.path.join(os.getcwd(), DATA_ROOT, 'audio')
SUMMARY_OUTPUT = os.path.join(os.getcwd(), DATA_ROOT, 'summaries')

In [3]:
url = 'https://www.youtube.com/watch?v=WZzKt-TC9WM' # Explainer video in the docs
out_dir = os.path.join(os.getcwd(), DATA_ROOT, 'argumentation-io')
json_schema_file = os.path.join(out_dir, 'schema.json')

In [4]:
json_schema = {
    '$schema': 'https://json-schema.org/draft/2020-12/schema',
    '$id': 'https://example.com/product.schema.json',
    'title': 'Product explainer summarizer',
    'description': 'A product explainer explainer.',
    'type': 'object',
    'properties': {
        'productName': {'description': 'The name of a product', 'type': 'string'},
        'targetUser': {
            'description': 'Description of target users of a product.',
            'type': 'object',
            'properties': {
                'segments': {
                    'description': "Explain the archetype user's job and characteristics.",
                    'items': {'type': 'string'},
                    'minItems': 1,
                    'uniqueItems': True
                },
                'needs': {
                    'description': "Explaining the archetype user's reasons for using the product.",
                    'items': {'type': 'string'},
                    'minItems': 1,
                    'uniqueItems': True
                }
            },
            'required': ['segments', 'needs']
        },
        'productFunction': {
            'description': 'What does this product do?',
            'type': 'object',
            'properties': {
                'features': {
                    'description': 'List of features provided by the product.',
                    'type': 'array',
                    'items': {
                        'type': 'object',
                        'properties': {
                            'featureName': {
                                'description': 'Name of the feature.',
                                'type': 'string'
                            },
                            'userExperience': {
                                'description': 'Description of the user experience for this feature.',
                                'type': 'string'
                            }
                        },
                        'required': ['featureName', 'userExperience']
                    },
                    'minItems': 1
                },
                'implementation': {
                    'description': 'Details on how the product is implemented.',
                    'type': 'object',
                    'properties': {
                        'developerTools': {
                            'description': 'Developer tools used for implementation.',
                            'type': 'array',
                            'items': {'type': 'string'},
                            'minItems': 1,
                            'uniqueItems': True
                        },
                        'platforms': {
                            'description': 'Platforms on which the product is available.',
                            'type': 'array',
                            'items': {'type': 'string'},
                            'minItems': 1,
                            'uniqueItems': True
                        }
                    },
                    'required': ['developerTools', 'platforms']
                }
            },
            'required': ['features', 'implementation']
        }
    },
    'required': ['productName', 'targetUser', 'productFunction']
}

In [5]:
with open(json_schema_file, 'w') as f:
    f.write(json.dumps(json_schema))

In [6]:
audio_filename = ytdlp_extract_audio(url, out_dir=AUDIO_OUTPUT)

Extracting audio from https://www.youtube.com/watch?v=WZzKt-TC9WM... Converting .mp3 to .wav 16-bit.


In [7]:
txt_filename = transcribe(
    audio_filename, 
    out_dir=out_dir,
    path_to_whisper_cpp=os.path.join(os.getcwd(), 'whisper.cpp')
)

Transcribing /Users/eddie/Dev/Alignment/public-calls-bot/data/audio/20230724_Argumentation.io Explainer.wav...


In [13]:
prompt_template = """You are an experienced and savvy product designer with many years of experience as a full-stack engineer.
This is a transcript from a product explainer video.

%s

Summarize the content in JSON. Do not repeat featureNames, segments, needs, or any other JSON array substructures. 
Highlight the key features of the app, who is likely the target user, and include relevant implementation and user experience details.

"""

In [14]:
prompt_file, summary_txt_file, summary_json_file  = summarize_key_features(
    transcript_filepath = txt_filename, 
    out_dir = out_dir, 
    json_schema_file = json_schema_file,
    prompt_template = prompt_template,
    path_to_llama_cpp = os.path.join(os.getcwd(), 'llama.cpp')
)

Summarizing /Users/eddie/Dev/Alignment/public-calls-bot/data/argumentation-io/20230724_Argumentation.io Explainer.txt...


Log start
main: build = 2933 (6aade19e)
main: built with Apple clang version 15.0.0 (clang-1500.3.9.4) for arm64-apple-darwin23.5.0
main: seed  = 1716141339
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/eddie/Dev/Alignment/public-calls-bot/llama.cpp/models/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length 

In [15]:
with open(summary_json_file, 'r') as f:
    summary_dict = json.loads(f.read())
JSON(summary_dict)

<IPython.core.display.JSON object>